In [24]:
import pandas as pd
import csv

file_path = "/home/xuxin/allCode/bxi_ros2_example/observations/observation@2025-08-06_14-21-18.csv"

freq=100
df=pd.read_csv(file_path, header=None)

timestamp_column = df.values[:, 0]
obs = df.values[:, 1:1+33]
obs_history = df.values[:,34:34+47*66]
action = df.values[:, 34+47*66:34+47*66+12]

commands = obs[:, 0:3]
dof_pos = obs[:, 3:15]
dof_vel = obs[:, 15:27]
obs_base_ang_vel = obs[:, 27:30]
obs_euler_angle = obs[:, 30:33]

time_length = timestamp_column.shape[0]


In [27]:
from humanoid_walk import humanoid_walk_onnx_Agent
import numpy as np

a=humanoid_walk_onnx_Agent("/home/xuxin/allCode/bxi_ros2_example/src/bxi_example_py_ankle/policy/model.onnx")
a.reset()
np.set_printoptions(formatter={'float': '{:.2f}'.format})

# 直接验证从obs_history推理是否一致
for i in range(time_length):
    obs_input = obs_history[i]
    obs_input = obs_input[None,:].astype(np.float32)
    input_feed = {"input": obs_input}

    actions_2 = np.squeeze(a.onnx_session.run(["output"], input_feed))
    actions_2 = np.clip(actions_2, -a.clip_action, a.clip_action)

    if not np.allclose(action[i], actions_2, atol=0.01):
        print(i)
        print(action[i])
        print(actions_2)


In [ ]:
a=humanoid_walk_onnx_Agent("/home/xuxin/allCode/bxi_ros2_example/src/bxi_example_py_ankle/policy/model.onnx")
a.reset()
np.set_printoptions(formatter={'float': '{:.2f}'.format})

for i in range(time_length):
    obs_group={
        "dof_pos":dof_pos[i],
        "dof_vel":dof_vel[i],
        "angular_velocity":obs_base_ang_vel[i],
        "euler_angle":obs_euler_angle[i],
        "commands":commands[i],
    }
    dof_target_2 = a.inference(obs_group)
    actions_2 = a.last_actions
    prop_obs_history = a.prop_obs_history.transpose().flatten() 

    # 验证obs_history构建是否一致
    if not np.allclose(obs_history[i], prop_obs_history, atol=0.0001):
        print("obs_history different",i)
        print(obs_history[i].reshape(66,47)[-1,:])
        print(prop_obs_history.reshape(66,47)[-1,:])

    # 验证action是否一致
    if not np.allclose(action[i], actions_2, atol=0.0001):
        print("action different",i)
        print(action[i])
        print(actions_2)


In [45]:
import onnxruntime as ort
import numpy as np

model_path = "/home/xuxin/allCode/bxi_ros2_example/src/bxi_example_py_ankle/policy/model.onnx"

providers = [
    'CUDAExecutionProvider',  # 优先使用GPU
    'CPUExecutionProvider'    # 回退到CPU
] if ort.get_device() == 'GPU' else ['CPUExecutionProvider']

# 启用线程优化配置
options = ort.SessionOptions()
options.intra_op_num_threads = 4  # 设置计算线程数
options.execution_mode = ort.ExecutionMode.ORT_SEQUENTIAL

# 创建推理会话
session = ort.InferenceSession(
    model_path,
    providers=providers,
    sess_options=options
)

# 预存输入输出信息
input_info = session.get_inputs()[0]
output_info = session.get_outputs()[0]

# 预分配输入内存（可选，适合固定输入尺寸）
input_buffer = np.zeros(
    input_info.shape,
    dtype=np.float32)

def inference_step(obs_data):
    # 使用预分配内存（如果适用）
    np.copyto(input_buffer, obs_data)  # 比直接赋值更安全
    
    # 极简推理（比原版快5-15%）
    return session.run(
        [output_info.name], 
        {input_info.name: input_buffer}
    )[0][0]  # 直接获取第一个输出的第一个样本


In [ ]:

a=humanoid_walk_onnx_Agent("/home/xuxin/allCode/bxi_ros2_example/src/bxi_example_py_ankle/policy/model.onnx")
a.reset()
np.set_printoptions(formatter={'float': '{:.2f}'.format})

for i in range(time_length):
    obs_group={
        "dof_pos":dof_pos[i],
        "dof_vel":dof_vel[i],
        "angular_velocity":obs_base_ang_vel[i],
        "euler_angle":obs_euler_angle[i],
        "commands":commands[i],
    }
    dof_target_2 = a.inference(obs_group)
    actions_2 = a.last_actions
    prop_obs_history = a.prop_obs_history.transpose().flatten() 

    # 验证obs_history构建是否一致
    if not np.allclose(obs_history[i], prop_obs_history, atol=0.0001):
        print("obs_history different",i)
        print(obs_history[i].reshape(66,47)[-1,:])
        print(prop_obs_history.reshape(66,47)[-1,:])

    # 验证action是否一致
    if not np.allclose(action[i], actions_2, atol=0.0001):
        print("action different",i)
        print(action[i])
        print(actions_2)